In [11]:
!pip -q install langchain tiktoken duckduckgo-search
!pip show langchain
!pip install openai

Name: langchain
Version: 0.0.200
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.7 MB/s eta 0:00:00


In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

## Importing Necessary Packages




In [3]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

## Setup Tools

In [4]:
search = DuckDuckGoSearchRun()
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
Writing the Prompt using ReACT

In [5]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [6]:
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [7]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [8]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )

        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)

        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [9]:
output_parser = CustomOutputParser()

## Set up LLM
Choose the LLM you want to use!

In [12]:
llm = OpenAI(temperature=0)

## Set up the Agent
We can now combine everything to set up our agent

In [13]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [14]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [15]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

In [16]:
agent_executor.run("How can I treat a swollen ankle?")



> Entering new  chain...
Thought: Swelling can be caused by a variety of conditions, so I should research the cause before treating the symptom.
Action: Search WebMD
Action Input: Swollen ankle

/usr/local/lib/python3.10/dist-packages/duckduckgo_search/compat.py:20: UserWarning: ddg is deprecated. Use DDGS().text() generator
  warnings.warn("ddg is deprecated. Use DDGS().text() generator")
/usr/local/lib/python3.10/dist-packages/duckduckgo_search/compat.py:22: UserWarning: parameter time is deprecated, use parameter timelimit
  warnings.warn("parameter time is deprecated, use parameter timelimit")
/usr/local/lib/python3.10/dist-packages/duckduckgo_search/compat.py:24: UserWarning: parameter page is deprecated, use DDGS().text() generator
  warnings.warn("parameter page is deprecated, use DDGS().text() generator")
/usr/local/lib/python3.10/dist-packages/duckduckgo_search/compat.py:26: UserWarning: parameter max_results is deprecated, use DDGS().text()
  warnings.warn("parameter max_results is deprecated, use DDGS().text()")




Observation:Weakness, swelling, and pain are the most common myositis symptoms. Myositis causes include infection, injury, autoimmune conditions, and drug side effects. Treatment of myositis varies according ... The outside of the knee and ankle; The top of the foot; Ganglion Cyst Symptoms. ... Swelling that may appear over time or suddenly. It may get smaller in size or even go away and come back. 6. Sudden, intense chest pain could mean the clot has broken off and caused a PE. Or it could be a sign that a clot in your artery gave you a heart attack. If so, you also might feel pain in your ... Gout. It's a form of arthritis that causes sudden pain, redness, swelling, and stiffness. It usually affects the large joint of the big toe, but it also can flare in the foot, ankle, or knees ... Lung congestion can also cause a dry, hacking cough or wheezing. Fluid and water retention. Less blood to your kidneys causes fluid and water retention, resulting in swollen ankles, legs, abdomen ...


'It is important to determine the cause of the swollen ankle before beginning treatment. Depending on the cause, treatment may include rest, elevation, compression, medications, or physical therapy. If the cause is an infection, it may require antibiotics. If the cause is an autoimmune condition, it may require immunosuppressant medications. It is important to speak with your doctor to determine the best course of treatment for your individual situation.'

### in Debug mode

In [17]:
langchain.debug = True

In [18]:
agent_executor.run("How can I treat a swollen ankle?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How can I treat a spained ankle?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "How can I treat a spained ankle?"
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can, but speaking as medical professional. You have access to the following tools:\n\nSearch WebMD: useful for when you need to answer medical and pharmalogical questions\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search WebMD]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nTho

'The best treatment for a sprain is to rest, ice, compress, and elevate the injury (RICE) and to use a cane, crutch, brace, or walking boot to help with mobility. Your doctor may also recommend physical therapy to strengthen the ankle and increase range of motion. Additionally, large scrapes should be kept moist with antibiotic ointment or moisture-enhancing bandages. To relieve metatarsalgia pain, the doctor may recommend staying off your feet and avoiding high impact activities for a while.'

In [19]:
langchain.debug = False

In [20]:
# Set up the base template
template_with_history = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a compasionate medical professional when giving your final answer. If the condition is serious advise they speak to a doctor.

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [21]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history"]
)

In [22]:
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)

In [23]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [24]:
from langchain.memory import ConversationBufferWindowMemory

memory=ConversationBufferWindowMemory(k=2)

In [25]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory
    )

In [26]:
agent_executor.run("How can I treat a swollen ankle?")



> Entering new  chain...

Thought: Swelling can be caused by a variety of conditions, so I should research the cause before treating the symptom.
Action: Search WebMD
Action Input: Swollen ankle

Observation:Weakness, swelling, and pain are the most common myositis symptoms. Myositis causes include infection, injury, autoimmune conditions, and drug side effects. Treatment of myositis varies according ... The outside of the knee and ankle; The top of the foot; Ganglion Cyst Symptoms. ... Swelling that may appear over time or suddenly. It may get smaller in size or even go away and come back. 6. Sudden, intense chest pain could mean the clot has broken off and caused a PE. Or it could be a sign that a clot in your artery gave you a heart attack. If so, you also might feel pain in your ... Gout. It's a form of arthritis that causes sudden pain, redness, swelling, and stiffness. It usually affects the large joint of the big toe, but it also can flare in the foot, ankle, or knees ... Lung

'It is important to speak to a doctor to determine the cause of the swollen ankle. Depending on the cause, treatment may include rest, elevation, compression, medications, physical therapy, or surgery.'

In [27]:
agent_executor.run("What meds could I take?")



> Entering new  chain...
Thought: I need to find out what medications could be used to treat a swollen ankle.
Action: Search WebMD
Action Input: Swollen ankle medications

Observation:These high blood pressure medications block formation of a hormone that causes blood vessels to narrow, so vessels relax. ACE inhibitors may cause these side effects: A dry, hacking cough that ... Weakness, swelling, and pain are the most common myositis symptoms. Myositis causes include infection, injury, autoimmune conditions, and drug side effects. ... Drugs. Many different medications ... Lung congestion can also cause a dry, hacking cough or wheezing. Fluid and water retention. Less blood to your kidneys causes fluid and water retention, resulting in swollen ankles, legs, abdomen ... Days 1-2: Recovery Begins. 3 /10. Your medical team will ask you to get up and start moving as soon as you can. You'll need to use crutches or a walker, because you have to keep your weight off ... The "classic triad" 

'Depending on the cause of the swollen ankle, treatment may include rest, elevation, compression, medications such as ACE inhibitors, physical therapy, or surgery. It is important to speak to a doctor to determine the best course of action.'

In [28]:
agent_executor.run("How long will it take to heal?")



> Entering new  chain...

Thought: I need to determine the cause of the swollen ankle before I can answer this question.
Action: Search WebMD
Action Input: Swollen ankle

Observation:Weakness, swelling, and pain are the most common myositis symptoms. Myositis causes include infection, injury, autoimmune conditions, and drug side effects. Treatment of myositis varies according ... The outside of the knee and ankle; The top of the foot; Ganglion Cyst Symptoms. ... Swelling that may appear over time or suddenly. It may get smaller in size or even go away and come back. 6. Sudden, intense chest pain could mean the clot has broken off and caused a PE. Or it could be a sign that a clot in your artery gave you a heart attack. If so, you also might feel pain in your ... Gout. It's a form of arthritis that causes sudden pain, redness, swelling, and stiffness. It usually affects the large joint of the big toe, but it also can flare in the foot, ankle, or knees ... Lung congestion can also caus

'The length of time it will take to heal depends on the cause of the swollen ankle. It is important to speak to a doctor to determine the best course of action and to ensure that the condition is properly treated.'